# M3E2

## Author: Raquel Aoki

Date: Spring 2021


In [1]:
!git clone https://github.com/raquelaoki/CompBioAndSimulated_Datasets.git

Cloning into 'CompBioAndSimulated_Datasets'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 54 (delta 27), reused 35 (delta 14), pack-reused 0
Receiving objects: 100% (54/54), 121.87 KiB | 904.00 KiB/s, done.
Resolving deltas: 100% (27/27), done.


In [2]:
import sys 
sys.path.insert(0,'src/')
#sys.path.insert(0,'bartpy/') #https://github.com/JakeColtman/bartpy
from CompBioAndSimulated_Datasets.simulated_data_multicause import *
#from bartpy.sklearnmodel import SklearnModel as bart
#from bartpy.features.featureimportance import feature_importance
#from bartpy.features.featureselection import SelectNullDistributionThreshold, SelectSplitProportionThreshold
#from bartpy.diagnostics.features import *

#from bartpy.sklearnmodel import SklearnModel as bart
#from bartpy.features.featureselection import SelectNullDistributionThreshold, SelectSplitProportionThreshold
#from bartpy.diagnostics.features import *
#from bartpy.features.featureimportance import feature_importance
#from sklearn.decomposition import PCA
#from sklearn.metrics import confusion_matrix,f1_score,roc_curve,roc_auc_score, accuracy_score




In [3]:
#sim_data = copula_simulated_data(s = 10)
#tr, u, y_continuous, y_binary = sim_data.get_data()
#effect_true, effect_obs, true_treat_obs_effect_01,true_treat_obs_effect= sim_data.get_true_coefs()
#sim_data.print_equation()

sdata_copula = copula_simulated_data()
X, y, y01,treatement_columns ,treatment_effects = sdata_copula.generate_samples()

Copula simulated data initialized!
... Treatments: (10000, 4)
... Confounders: (10000, 10)
Data Simulation Done!


In [4]:
sdata_gwas = gwas_simulated_data()
X, y, y01, treatement_columns, treatment_effects  = sdata_gwas.generate_samples()

GWAS simulated data initialized!
... Treatments:  10
... Confounders:  90
 Data Simulation Done!


In [7]:
def Find_Optimal_Cutoff(target, predicted):
    """ Find the optimal probability cutoff point for a classification model related to event rate
    Parameters
    ----------
    target : Matrix with dependent or target data, where rows are observations
    predicted : Matrix with predicted data, where rows are observations
    Returns
    -------
    list type, with optimal cutoff value
    https://stackoverflow.com/questions/28719067/roc-curve-and-cut-off-point-python
    """
    fpr, tpr, threshold = roc_curve(target, predicted)
    i = np.arange(len(tpr))
    roc = pd.DataFrame({'tf': pd.Series(tpr - (1 - fpr), index=i), 'threshold': pd.Series(threshold, index=i)})
    roc_t = roc.iloc[(roc.tf - 0).abs().argsort()[:1]]
    return list(roc_t['threshold'])

In [101]:

#y = y.astype('float')

T = X[:,col]
X1 = np.delete(X,col,1)

pca = PCA(n_components=10)
X3 = np.concatenate((T,pca.fit_transform(X1)),axis = 1)

model = bart(n_samples=1000, n_burn=10, n_trees=25, store_in_sample_predictions=False, n_jobs=1) # Use default parameters
#model.fit(X3, y) # Fit the model
#y_ = model.predict(X3)

#...Training set: F1 -  0.5418502202643172 25
#...... confusion matrix:  [347 232 192 229]

In [92]:
thhold = Find_Optimal_Cutoff(y,y_)
y_01 = [0 if item < thhold else 1 for item in y_]

#random 
prob = sum(y)/len(y)
y_random = [np.random.binomial(1,prob) for item in y_]

print('Original:', np.sum(y)/len(y))

print('\n...Training set: F1 - ',f1_score(y,y_01))
print('...Training set: acc - ',accuracy_score(y,y_01))
print('...... confusion matrix: \n',confusion_matrix(y,y_01))

print('\n...Random set: F1 - ',f1_score(y,y_random))
print('...Random set: acc - ',accuracy_score(y,y_random))
print('...... confusion matrix: \n',confusion_matrix(y,y_random))

NameError: name 'y_' is not defined

In [ ]:
#stats.invgamma(1,1).rvs(3, random_state = 99)/max(stats.invgamma(1,1).rvs(3, random_state = 99))

In [ ]:
col

In [125]:
from sklearn.linear_model import LogisticRegression
T = X[:,col]
X1 = np.delete(X,col,1)

pca = PCA(n_components=20)
X3 = np.concatenate((T,pca.fit_transform(X1)),axis = 1)


clf = LogisticRegression(C=1e9, random_state=0, penalty='l2', class_weight = {0:4,1:7}).fit(X3, y)
y01lr = clf.predict(X3)

y01b = clf.predict_proba(X3)
thhold = Find_Optimal_Cutoff(y,y01b[:,1])
y01bb = [0 if item < thhold else 1 for item in y01b[:,1]]

print('\n...Training set: F1 - ',f1_score(y,y01lr))
print('...Training set: acc - ',accuracy_score(y,y01lr))
print('...... confusion matrix: \n',confusion_matrix(y,y01lr))

print('\n...Training set: F1 - ',f1_score(y,y01bb))
print('...Training set: acc - ',accuracy_score(y,y01bb))
print('...... confusion matrix: \n',confusion_matrix(y,y01bb))

print(clf.coef_[0][0:len(col)],'\n',tc[col])
print((y.sum()/len(y)))

#random 
prob = sum(y)/len(y)
y_random = [np.random.binomial(1,prob) for item in y]
print('\n...Random set: F1 - ',f1_score(y,y_random))
print('...Random set: acc - ',accuracy_score(y,y_random))
print('...... confusion matrix: \n',confusion_matrix(y,y_random))


...Training set: F1 -  0.523366097603035
...Training set: acc -  0.4472
...... confusion matrix: 
 [[1437 4754]
 [ 774 3035]]

...Training set: F1 -  0.4559680182752713
...Training set: acc -  0.5237
...... confusion matrix: 
 [[3241 2950]
 [1813 1996]]
[-0.050884   -0.03545732 -0.07816463 -0.0301015   0.00991149] 
 [-0.62465696  0.67856941 -0.31181707 -1.00357935  0.47113778]
0.3809

...Random set: F1 -  0.3821622336906179
...Random set: acc -  0.5331
...... confusion matrix: 
 [[3887 2304]
 [2365 1444]]


In [88]:
npr.binomial(2,[0.5, 0.5,.5])

array([0, 1, 1])